In [1]:
import pandas as pd
import numpy as np
import ast

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.abspath(''))))

from yumspeak_ml.params import *
from yumspeak_ml.ml_logic.data import clean_restaurant_data

In [2]:
len(CATS_TO_REMOVE)

307

In [3]:
df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/all_restaurants_dataset.csv').drop(columns=['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
df.head()

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
0,ChIJYR6HCJ4Z2jERChIQxjl2MhI,Red House Seafood Grand Copthorne,1110,Seafood restaurant,"['Seafood restaurant', 'Asian restaurant', 'Di...",4.2,"392 Havelock Rd, Level 2 Grand Copthorne Water...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipNp...
1,ChIJP2EDPqAZ2jERJKipSQq7qY8,Red House Seafood Nanyang (Clarke Quay),730,Seafood restaurant,['Seafood restaurant'],4.1,"3C River Valley Road 01-02/03, The Cannery, 17...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipMe...
2,ChIJSW7NnhAZ2jERj8p3W_nnhGE,Red House Seafood at Esplanade,186,Seafood restaurant,"['Seafood restaurant', 'Chinese restaurant']",4.4,"8 Raffles Ave., #01-14 / 16 Esplanade, Singapo...",https://www.google.com/maps/place/Red+House+Se...,['https://lh5.googleusercontent.com/p/AF1QipOJ...
3,ChIJNxLu-EAb2jER_a4Y4ahn8TU,Red Sparrow,496,Vietnamese restaurant,"['Vietnamese restaurant', 'Asian restaurant']",4.0,"11 Dempsey Rd, #01-18, 249673, Singapore 249673",https://www.google.com/maps/place/Red+Sparrow/...,['https://lh5.googleusercontent.com/p/AF1QipO8...
4,ChIJIds6rnUZ2jERPx6E0UCf6HM,Red Star Restaurant,3725,Cantonese restaurant,"['Cantonese restaurant', 'Chinese restaurant',...",3.7,"Block 54 Chin Swee Rd, #07-23, Singapore 160054",https://www.google.com/maps/place/Red+Star+Res...,['https://lh5.googleusercontent.com/p/AF1QipN-...


In [5]:
#drop duplicates with subset 'place_id', 'name', 'reviews', 'address']
df = df.drop_duplicates(subset=['place_id', 'name', 'reviews', 'address'])

# remove irrelevant place based on main_categories
cats_to_remove = [cat.lower() for cat in CATS_TO_REMOVE]
mask = df['main_category'].str.lower().isin(cats_to_remove)
df = df[~mask]

# remove irrelevant place based by name
#df = df[~df['name'].isin(NAME_TO_DROP)]
df = df[~df['place_id'].isin(PLACE_ID_TO_DROP)]

# fill na in main_category and categories(as ['unknown'])
df['main_category'].fillna('unknown', inplace=True)
df['categories'] = df['categories'].fillna("['unknown']").apply(ast.literal_eval)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4946 entries, 0 to 10093
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       4946 non-null   object 
 1   name           4946 non-null   object 
 2   reviews        4946 non-null   int64  
 3   main_category  4946 non-null   object 
 4   categories     4946 non-null   object 
 5   main_rating    4946 non-null   float64
 6   address        4916 non-null   object 
 7   link           4946 non-null   object 
 8   review_photos  4946 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 386.4+ KB


In [6]:
df.to_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/review_scraper_data/restaurant_data/final_restaurant_b1.csv', index=False, header=True)

In [8]:
contains_pte_ltd = df['name'].str.contains('Pte Ltd', case=False, na=False)
contains_pte_ltd.sum()

59

In [9]:
unknown_main_cat = df['main_category'] == 'unknown'
unknown_main_cat.sum()

68

In [10]:
no_rating = df['main_rating'] == 0
no_rating.sum()

244

In [11]:
restaurant = df['main_category'].str.contains('restaurant', case=False, na=False)
restaurant.sum()

4150

In [16]:
to_remove = df[(~restaurant & contains_pte_ltd & no_rating) | unknown_main_cat].reset_index(drop=True)
to_remove.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       69 non-null     object 
 1   name           69 non-null     object 
 2   reviews        69 non-null     int64  
 3   main_category  69 non-null     object 
 4   categories     69 non-null     object 
 5   main_rating    69 non-null     float64
 6   address        64 non-null     object 
 7   link           69 non-null     object 
 8   review_photos  69 non-null     object 
dtypes: float64(1), int64(1), object(7)
memory usage: 5.0+ KB


In [21]:
to_remove[30:]

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
30,ChIJ3Tcl0IwZ2jER7_dBq_HSCbA,Uni-rise Pte Ltd,0,unknown,[unknown],0.0,"400 Orchard Rd, Singapore 238875",https://www.google.com/maps/place/Uni-rise+Pte...,[]
31,ChIJ1239LW0Z2jERjAM826R9LvI,Amò,1,unknown,[unknown],4.0,"57 Tras St, Singapore 078996",https://www.google.com/maps/place/Am%C3%B2/dat...,[]
32,ChIJa1-WpnIZ2jERHQAr226M9eo,Amò,0,unknown,[unknown],0.0,"18 Tg Pagar Rd, Singapore 088441",https://www.google.com/maps/place/Am%C3%B2/dat...,[]
33,ChIJaQmIU40Z2jER0iORlK_9C3w,Anjappar,0,unknown,[unknown],0.0,NaN,https://www.google.com/maps/place/Anjappar/dat...,[]
34,ChIJLasDDPsa2jERHRw5n8EXprY,Atout Pte. Ltd.,0,unknown,[unknown],0.0,"55 W Coast Rd, Singapore 127365",https://www.google.com/maps/place/Atout+Pte.+L...,[]
35,ChIJq9L6dgAZ2jERfx1hmXmMcSQ,CHUAN Seafood Restaurant@Robertson Quay,6,unknown,[unknown],4.8,"86 Robertson Quay, #01-02, Singapore 238245",https://www.google.com/maps/place/CHUAN+Seafoo...,['https://lh5.googleusercontent.com/p/AF1QipM7...
36,ChIJKdihA68Z2jER07IZ_I3hEVI,Colony Capital Asia Pacific Pte Ltd,0,unknown,[unknown],0.0,NaN,https://www.google.com/maps/place/Colony+Capit...,[]
37,ChIJ-doSHDnyloAR3j9w6munIlM,Curry Village,0,unknown,[unknown],0.0,"Yosemite Valley, CA 95389, USA",https://www.google.com/maps/place/Curry+Villag...,[]
38,ChIJAR3XlcbSAzQR-PJpUp1NPd0,Dongqu Subdistrict,0,unknown,[unknown],0.0,"Zhongshan, Guangdong Province, China, 528403",https://www.google.com/maps/place/Dongqu+Subdi...,[]
39,ChIJv-yIJjt1ZjURCqXRAIlYVzo,Dong-gu,0,unknown,[unknown],0.0,"Daegu, South Korea",https://www.google.com/maps/place/Dong-gu/data...,[]


In [22]:
df['main_category'] = df['main_category'].apply(lambda x: x.replace(' restaurant', ''))

In [25]:
df.main_category.unique()

array(['Seafood', 'Vietnamese', 'Cantonese', 'Indian', 'Restaurant',
       'Dim sum', 'Cafe', 'Singaporean', 'Indonesian', 'Halal',
       'Fine dining', 'Modern European', 'Asian fusion', 'Thai', 'Nyonya',
       'Chinese', 'French', 'Hot pot', 'Italian', 'Vegetarian',
       'Mediterranean', 'Meat dish', 'Pizza', 'Family', 'Korean',
       'Japanese', 'Grill', 'Steak house', 'Sushi', 'Hamburger', 'Asian',
       'Bar', 'Traditional', 'Tapas', 'Mexican', 'Chinese noodle',
       'Pakistani', 'Sichuan', 'Turkish', 'Modern French', 'Salad shop',
       'Seafood donburi', 'Tex-Mex', 'Chicken', 'South Indian', 'Andhra',
       'Lebanese', 'Fish and seafood', 'Izakaya', 'Authentic Japanese',
       'Modern izakaya', 'Conveyor belt sushi', 'Unagi', 'Oyster bar',
       'Bistro', 'Steamboat', 'Indian Muslim Restaurant', 'Malaysian',
       'Nasi', 'Brunch', 'Bakso', 'Breakfast', 'Fried chicken takeaway',
       'Padang', 'Ramen', 'Cha chaan teng (Hong Kong-style cafe)',
       'Tempura donb

In [ ]:
# 'Japanese sweets', 'Gastropub', 'Buffet', 'Karaoke bar', 'Delivery Chinese', 'Diner', 'Coffee stand',
# 'Dessert shop', 'Delivery Chinese', 'Coffee roasters', 'Playground', 'Indoor playground', 'Sushi takeaway',
# 'Beer garden', Chinese bakery', 'Chinese tea house', 'Wholesale bakery', 'Food manufacturing supply',
# 'Pizza takeaway', 'Self service', 'Tea house', 'Chocolate artisan', 'Food and drink', 'Sundae', 'Chocolate shop', 'Kitchen supply store',
# 'Fruit and vegetable wholesaler', 'Stationery store', 'Tea and coffee shop', 'Butcher shop deli',
# 'Steamed bun shop'

In [26]:
df[df['main_category'] == 'Restaurant']

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos
6,ChIJSaWaOAUZ2jERO4wmur1vU_U,Regal Golden Steamboat,2,Restaurant,[Restaurant],4.5,"1080 Lower Delta Rd, Singapore 169311",https://www.google.com/maps/place/Regal+Golden...,['https://lh5.googleusercontent.com/p/AF1QipOf...
9,ChIJPZE_io4Z2jERhBAeYar2_ZE,Rempah by Chilli Padi,54,Restaurant,[Restaurant],3.4,"Singapore General Hospital (SGH) Block 2/3, 1 ...",https://www.google.com/maps/place/Rempah+by+Ch...,['https://lh5.googleusercontent.com/p/AF1QipPc...
15,ChIJ6eCqdV0Z2jERGTysrBvGX_M,Restaurant Chaleur,54,Restaurant,[Restaurant],5.0,"77A Neil Rd, Level 2, Singapore 088903",https://www.google.com/maps/place/Restaurant+C...,['https://lh5.googleusercontent.com/p/AF1QipOV...
17,ChIJ99ae4CcZ2jERhNdU8AYgcsg,Restaurant Euphoria,159,Restaurant,"[Restaurant, Fine dining restaurant]",4.3,"76 Tras St, Singapore 079015",https://www.google.com/maps/place/Restaurant+E...,['https://lh5.googleusercontent.com/p/AF1QipM6...
20,ChIJ7-udqZQZ2jERYa5K9CfhBDk,Restaurant Imbue,42,Restaurant,[Restaurant],4.7,"32 Keong Saik Rd., Singapore 089137",https://www.google.com/maps/place/Restaurant+I...,['https://lh5.googleusercontent.com/p/AF1QipNx...
...,...,...,...,...,...,...,...,...,...
10057,ChIJA4FMt-kZ2jERghZLxDYONFM,Hongdae BBQ food stall,0,Restaurant,[Restaurant],0.0,"Smith Street, Chinatown Food Street, Singapore...",https://www.google.com/maps/place/Hongdae+BBQ+...,[]
10068,ChIJXYrMYNA0K4gRwtEJIQ7lZ6s,Hooters Downtown Toronto,2103,Restaurant,"[Restaurant, American restaurant, Chicken wing...",3.6,"280 Adelaide St W, Toronto, ON M5V 1P6, Canada",https://www.google.com/maps/place/Hooters+Down...,['https://lh5.googleusercontent.com/p/AF1QipNG...
10073,ChIJQ6cPiSMZ2jERYP9lJruuJFA,Hopscotch (Gardens by the Bay),462,Restaurant,"[Restaurant, Cocktail bar]",4.5,"18 Marina Gardens Dr, #01-21 Bayfront Plaza (N...",https://www.google.com/maps/place/Hopscotch+(G...,['https://lh5.googleusercontent.com/p/AF1QipOR...
10081,ChIJvRlPUrwP2jERvGvJbVmKm9Q,House Of Kebabs,25,Restaurant,[Restaurant],3.6,"762 Jurong West Street 75, Singapore 640762",https://www.google.com/maps/place/House+Of+Keb...,['https://lh5.googleusercontent.com/p/AF1QipOa...
